МФТИ ФИВТ: Курс Машинное Обучение (осень, 2016), Арсений Ашуха, ars.ashuha@gmail.com

In [ ]:
%pylab inline 

# Проверьте установлин ли у вас grapviz "dot -h" в терминале
# Обновите pydot "sudo pip2 install pydot --upgrade" в терминале

import warnings
warnings.simplefilter("ignore")

<h1 align="center">Course Rulles</h1> 

Курс будет состоять из 12 Лекций и 4 Гостевых лекций (от лучших в стране специалистов ;)). 

**Домашнее задание**
- Будет выдаваться после каждого семинара, иногда это будут контексты
- Задание необходимо сдать в течении 6 дней после семинара
- Задание оценивается  1, если вы его сделали достаточно хорошо
- Задание оценивается  0, если вы не выполнили задание или сделали много ГРУБЫХ ошибок 
- Задание оценивается -1, если вы его списали (и тому кто и тому у кого)
- Задания, сданные после дедлайна, не проверяются
- **PS**: Каждый 0 или -1 это доп. вопрос на экзамене, который идет в итоговую оценку за экзамен. 

**Контексты**

- Будет проведено 4 контеста 
- Задание необходимо сдать в течение 6 дней после семинара
- Вы получаете 1 балл, если побили бейзлайн и коротко текстом описали решение, 0 - если нет
- Бонусы получают первые три человека в каждой в группе

**Экзамен**

- Будет экзамен, автоматов не будет
- Будет список вопросов 
- Будет теор. минимум

**Оценка**

Финальная оценка будет вычисляться, как
    $$mark = [exam\_creidts > c_0] \cdot (c_1 \cdot hw\_credits + c_2 \cdot contest\_credits + c_3 \cdot exam\_creidts)$$
коэффициенты будут объявлены позже

<h1 align="center">In the previous series</h1> 

### Linear Model Theoretical Background

Мы хотим научиться предсказывать какую-то величину $y_i$ по каким-то признакам $x_i$.

Линейные модели - это набор методов, которые предполагают, что целевое значение ($y_i$) может быть восстановлено как линейная комбинация признаков ($x_i$). 

$$\hat{y}_i(x_i, w) = x_i \cdot w^T$$

<img src="img/linear_model.png">

Для обучения модели, вводится функция потерь $L_i(\hat{y}_i(x_i), y_i)$, к примеру:
- Для регрессии:
$$L_i(\hat{y}_i(x_i, w), y_i) = (\hat{y}_i(x_i, w) - y_i)^2$$ 
- Для классификации: 
$$L_i(\hat{y}_i(x_i, w), y_i) = \sum_{y_i} y_i \log \hat{y}_i(x_i, w)$$
Целевой функцией является сумма ошибок по всем объектам тренировочной выборки

$$L(X, Y, w) = \sum_{x_i} L_i(y_i(x_i, w), y_i) \rightarrow \min_w$$

Оптимизируем функцию:
- Если данных много -- стохастические, градиентные методы 

$$w_k = w_{k-1} - step\_length \cdot \frac{dL}{dw}$$

- Если данных мало -- методы второго порядка, использующие гессиан (HFN, LBFGS, ...)

### Trees Theoretical Background

Мы решаем всю ту-же задачу, только теперь хотим предсказать неизвестное значение не как линейную комбинацию признаков, а как дерево, в каждом узле которого есть какие-то правила. 

<img src="img/tree_model.png">

<center> Предсказание смерти пассажиров Титаника. </center>

Для обучения модели обычно используется жадная стратегия, которая стремится как можно сильнее разделить классы (снизить ошибку предсказания) при построении каждого листа дерева. К примеру, можно строить пороговые функции (x_i > w_i) минимизируя следующие функции:
- Для задачи классификации -- ответ, номер класса с большей вероятностью
$$ H(Y_{leaf}) = |leaf\_left|\sum_{y \in Y_{leaf\_left}} p(y) log~p(y) + |leaf\_right|\sum_{y \in Y_{leaf\_right}} p(y) log~p(y)$$
- Для задачи регрессии, ответ среднее значение в листе
$$ MSE(Y_{leaf}) = |leaf\_left| \sum_{y \in Y_{leaf_left}} (E(y) - y)^2 + |leaf\_right| \sum_{y \in Y_{leaf_right}} (E(y) - y)^2$$

### Some Facts about a Trees and a Linear models

- Linear Model
    - Слишком простые
    - Не сильно меняются при небольшом изменении выборки
- Tree Model
    - Слишком сложные -- легко закодировать всю выборку деревом
    - Сильно меняются при небольшом изменении выборки

### Загрузим датасет титаник

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [ ]:
titanic = pd.read_csv('./data/train.csv')[['Survived', 'Pclass', 'Sex', 'Age', 'Fare']]
titanic.head()

In [ ]:
sex_encoder = LabelEncoder()
titanic.Sex = sex_encoder.fit_transform(titanic.Sex)
features = ['Pclass', 'Sex', 'Age', 'Fare']
X, y = titanic[features].values, titanic.Survived.values

X = np.nan_to_num(X)

### Обучим диненую модель на разных подвыборках, похожи-ли коэфиценты?

In [ ]:
from sklearn.linear_model import LogisticRegression

for n_items in [200, 500]:
    reg = LogisticRegression().fit(X[:n_items], y[:n_items])
    print 'n_items', n_items, 'w = ',', '.join(map(lambda x: '%.4f' % x, reg.coef_[0]))

### Обучем деревья на тех-же подвыборках, похожи-ли деревья?

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz

tree1 = DecisionTreeClassifier().fit(X[:200], y[:200])
tree2 = DecisionTreeClassifier().fit(X[:500], y[:500])

Если не работает pydot, то можно установить pydotplus и заменить все включения. Дополнительно необходимо установить graphviz для отображения дерева.
Подробности установки можно найти на http://graphviz.org/

In [ ]:
import pydot 
from StringIO import StringIO
from IPython.display import Image  

dot_data = StringIO()  
export_graphviz(tree1, out_file=dot_data)  
graph = pydot.graph_from_dot_data(dot_data.getvalue())[0]
Image(graph.create_png())  

In [ ]:
dot_data = StringIO()  
export_graphviz(tree2, out_file=dot_data)  
graph = pydot.graph_from_dot_data(dot_data.getvalue())[0]
Image(graph.create_png()) 

Давайте разберемся, что происходит

<h1 align="center">Bias-variance tradeoff</h1> 

### Teoretical Background

#### Какя природа бывает у ошибок

<img src="img/bv_darts.png", width=400>

#### Как это отражается на модели

<img src="img/bv_model_complex.png">

### Teoretical Task

$$\mathrm{E}\Big[\big(y - \hat{f}(x)\big)^2\Big] = \mathrm{Bias}\big[\hat{f}(x)\big]^2 + \mathrm{Var}\big[\hat{f}(x)\big] + \sigma^2 \\$$

---------
$$ \mathrm{Bias}\big[\hat{f}(x)\big] = \mathrm{E}\big[\hat{f}(x) - f(x)\big] $$


and

$$\mathrm{Var}\big[\hat{f}(x)\big] = \mathrm{E}[\hat{f}(x)^2] - \mathrm{E}[\hat{f}(x)]^2$$

----------


<img src="img/bvts.png">

<h1 align="center">Bagging</h1> 

### Theoretical Background

- Усреднение моделей
- Работает, потому что не ухудшает смещение, а разброс уменьшается линейно (если алгоритмы не коррелированы)
- Нужны модели с маленьким смещением (Какие???)
- Нужны некоррелированные модели

Help: Раздел 3.2 https://github.com/esokolov/ml-course-msu/blob/master/ML15-spring/lecture-notes/Sem04_ensembles.pdf  

### Смещение для беггинга

<img src="img/bag_mu1.png">

<img src="img/bag_mu2.png">

### Разброс для беггинга

<img src="img/bag_var1.png">

<img src="img/bag_var2.png">

<img src="img/bag_var3.png">

### Practical Task Model Сorrelation (Linear and Trees Bagging)

### Read data

In [ ]:
!sh ./get_data.sh

In [ ]:
adult = pd.read_csv(
    './data/adult.data', 
    names=[
        "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
        "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
        "Hours per week", "Country", "Target"], 
    header=None, na_values="?")

In [ ]:
adult.head(2)

In [ ]:
adult = pd.get_dummies(adult)

In [ ]:
adult.head(2)

In [ ]:
adult["Target"] = adult["Target_ >50K"]

In [ ]:
adult.columns[:-3]

In [ ]:
X, y = adult[adult.columns[:-3]].values, adult[adult.columns[-1]].values

### Model correlation

In [ ]:
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import BaggingClassifier

### Бегинг на лог-регрессиях 

In [ ]:
from sklearn.linear_model import LogisticRegression

clf1 = BaggingClassifier(base_estimator=LogisticRegression(), n_estimators=100)
scores1 = cross_val_score(clf1, X, y, cv=4, n_jobs=4)

print scores1

### Бегинг на девевьях

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf2 = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100)
scores2 = cross_val_score(clf2, X, y, cv=4, n_jobs=4)

print scores2

### Натренируем модели и вытащим базовые алгоритмы

In [ ]:
X_train, y_train, X_test, y_test = X[:20000], y[:20000], X[20000:], y[20000:]

clf1 = clf1.fit(X_train, y_train)
clf2 = clf2.fit(X_train, y_train)

In [ ]:
X_test.shape

In [ ]:
clf1.estimators_[0].predict(X_test)

### Посмотрим на корреляцию предсказаний

In [ ]:
pred1 = [clf.predict(X_test) for clf in clf1.estimators_]
pred2 = [clf.predict(X_test) for clf in clf2.estimators_]

In [ ]:
from scipy.stats import pearsonr

In [ ]:
pearsonr(pred1[0], pred1[1])[0]

In [ ]:
pearsonr(pred2[0], pred2[1])[0]

In [ ]:
A = np.zeros((len(clf1.estimators_), len(clf1.estimators_)))

for i in range(len(clf1.estimators_)):
    for j in range(len(clf1.estimators_)):
        A[i, j] = pearsonr(pred1[i], pred1[j])[0]

In [ ]:
pylab.imshow(A, interpolation='none', cmap='hot')

In [ ]:
A = np.zeros((len(clf1.estimators_), len(clf1.estimators_)))

for i in range(len(clf1.estimators_)):
    for j in range(len(clf1.estimators_)):
        A[i, j] = pearsonr(pred2[i], pred2[j])[0]

In [ ]:
pylab.imshow(A, interpolation='none', cmap='hot')

###  Есть-ли Overfitting с увиличением числа деревьев и сложности?

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
%%time

for n_estimators in [10, 40, 100, 200, 600, 1000]:
    clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=n_estimators, n_jobs=4)
    clf = clf.fit(X_train, y_train)
    train_acc, test_acc = accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test)
    print 'n_estimators = %4s train_acc = %4s test_acc = %4s' %(n_estimators, train_acc, test_acc)

In [ ]:
%%time

for n_estimators in [10, 40, 100, 200, 600, 1000]:
    clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=7), n_estimators=n_estimators, n_jobs=4)
    clf = clf.fit(X_train, y_train)
    train_acc, test_acc = accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test)
    print 'n_estimators = %4s train_acc = %4s test_acc = %4s' %(n_estimators, train_acc, test_acc)

In [ ]:
%%time

for n_estimators in [10, 40, 100, 200, 600, 1000]:
    clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=14), n_estimators=n_estimators, n_jobs=4)
    clf = clf.fit(X_train, y_train)
    train_acc, test_acc = accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test)
    print 'n_estimators = %4s train_acc = %4s test_acc = %4s' %(n_estimators, train_acc, test_acc)

<h1 align="center">Resume</h1> 

**Смещение и разброс**:
    - Ошибки бывают разные
    - Ошибку можно разложить на составляющие части, чтобы лучше понимать, что происходит
    - Это может помочь нам придумать хорошие алгоритмы и понимать, как такие алгоритмы будут работать

**Bagging**:
    - Одна из лучших техник для построения алгоритмов ML
    - Линейно уменьшает Разброс и не уменьшает Смещение 
    - Слабое переобучение
    - НО переобучение ЕСТЬ -- от сложности одного алгоритма, лучше все же немного обрезать деревья

**Что почитать**:
- Hastie, The Elements of Statistical Learning, https://goo.gl/k3wfEU
    - 2.9 Model Selection and the Bias–Variance Tradeoff 
    - 15 Random Forests
- Соколов, Семинары по композиционным методам, https://goo.gl/sn8RyJ
- Andrew Ng, Bias vs. Variance, https://goo.gl/1ISZ6Y 